# Introduction to LANGCHAIN

initializing gemini api keys

In [2]:
import os
from dotenv import load_dotenv

# load all variables from .env file
load_dotenv()

google_key = os.environ.get("GOOGLE_API_KEY")

LangChain is a framework for building applications with LLMs (Large Language Models).
Think of it like a toolbox that helps you:

* Connect an LLM (like GPT, LLaMA, etc.)
* Chain steps together (e.g., prompt → model → reasoning → output)
* Add memory (so it “remembers” conversations)
* Integrate external knowledge (databases, PDFs, APIs, etc.)
* Build Agents that can make decisions and use tools.

* **ChatGoogleGenerativeAI** : Gemini Connector, helps langchain connect with gemini
* **LLMChain** : A chain in langchain is a simple workflow and LLMChain is the simplest. It takes a prompt and sends to llm and returns the result.
    - temperature: controls the creativity of llm (higher = more cretive)
* **PromptTemplate**: It is a reusable template/blueprint for prompts.

Here we created a simple chain 
Inputs -> Prompt -> LLM -> Ouptput

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

llm = ChatGoogleGenerativeAI(
    model = "gemini-1.5-flash",
    api_key = google_key,
    temperature = 0.7
)

#create a prompt template with a placeholder for an animal
prompt = PromptTemplate.from_template("Suggest a name for a {animal}.")
#wrap it into a chain
chain = LLMChain(
    llm = llm,
    prompt = prompt
)
#run the chain
print(chain.invoke({"animal": "dog"}))


{'animal': 'dog', 'text': "**For a Male Dog:**\n\n* **Classic:**  Duke, Charlie, Cooper, Jack, Buddy\n* **Unique:**  Arlo, Finn, Jasper,  Reggie,  Ziggy\n* **Strong:**  Ranger,  Thor,  Magnus,  Atlas,  Bear\n\n**For a Female Dog:**\n\n* **Classic:**  Lucy, Bella, Daisy, Sophie, Chloe\n* **Unique:**  Willow,  Juniper,  Hazel,  Piper,  Rhea\n* **Sweet:**  Sugar,  Honey,  Lily,  Rose,  Buttercup\n\n\n**Based on Appearance:**\n\n* **If they're fluffy:** Cloud,  Cotton,  Snowball\n* **If they're small:**  Peanut,  Pip,  Squirt\n* **If they're big:**  Titan,  Giant,  Mountain\n\n\nTo help me give you a better suggestion, tell me:\n\n* **What is the dog's breed or appearance?**\n* **What is the dog's personality like?** (Playful, calm, energetic, etc.)\n* **What kind of name are you looking for?** (Cute, strong, sophisticated, etc.)"}


In [7]:
prompt2 = PromptTemplate.from_template("Suggest a name for a {company} that makes {product} and a tagline.")

chain2 = LLMChain(
    llm = llm,
    prompt = prompt2,
)

In [8]:
output = chain2.invoke({"company":"Chemical" , "product":"soap"})

In [ ]:
print(output)

{'company': 'Chemical', 'product': 'soap', 'text': "Here are a few name and tagline suggestions for a chemical that makes soap:\n\n**Option 1 (Emphasizing ease of use):**\n\n* **Name:**  SaponifyEasy\n* **Tagline:**  Soapmaking simplified.\n\n**Option 2 (Highlighting effectiveness):**\n\n* **Name:**  SudsSurge\n* **Tagline:** The power of soap, unleashed.\n\n**Option 3 (Focusing on natural ingredients - if applicable):**\n\n* **Name:**  BotanicaBase\n* **Tagline:**  Naturally superior soapmaking.\n\n\n**Option 4 (More scientific and sophisticated):**\n\n* **Name:**  LyeLuxe (if it's lye-based) or  EsterEase (if it's ester-based)\n* **Tagline:**  Precision soapmaking, perfected.\n\n\n**Option 5 (Short and catchy):**\n\n* **Name:**  SoapStart\n* **Tagline:**  Your soapmaking journey begins here.\n\n\nWhen choosing, consider:\n\n* **Target audience:**  Are you selling to hobbyists, large manufacturers, or both?\n* **Key selling points:** What makes your chemical unique or superior?\n* **B

In [12]:
print(output['text'])

Here are a few name and tagline suggestions for a chemical that makes soap:

**Option 1 (Emphasizing ease of use):**

* **Name:**  SaponifyEasy
* **Tagline:**  Soapmaking simplified.

**Option 2 (Highlighting effectiveness):**

* **Name:**  SudsSurge
* **Tagline:** The power of soap, unleashed.

**Option 3 (Focusing on natural ingredients - if applicable):**

* **Name:**  BotanicaBase
* **Tagline:**  Naturally superior soapmaking.


**Option 4 (More scientific and sophisticated):**

* **Name:**  LyeLuxe (if it's lye-based) or  EsterEase (if it's ester-based)
* **Tagline:**  Precision soapmaking, perfected.


**Option 5 (Short and catchy):**

* **Name:**  SoapStart
* **Tagline:**  Your soapmaking journey begins here.


When choosing, consider:

* **Target audience:**  Are you selling to hobbyists, large manufacturers, or both?
* **Key selling points:** What makes your chemical unique or superior?
* **Brand personality:** Do you want to project a scientific, natural, or easy-going image?

# Connecting two chains

* chain1 -> chain2
* chain1: write a story based on a {topic}
* chain2: Takes that story and generate a cachy title for it.

In [ ]:
from langchain.chains import SequentialChain

#chain 1: Story Generator
prompt1 = PromptTemplate.from_template(
    "Write a story in 1-2 sentences about {topic}."
)
chain1 = LLMChain(
    llm = llm,
    prompt = prompt1,
    output_key = "story"
)

#chain1 generates a story and stores it in key "story"

prompt2 = PromptTemplate.from_template(
    "Generate one creative title for this story: \n{story}."
)
chain2 = LLMChain(
    llm = llm,
    prompt = prompt2,
    output_key = "title"
)

#create a sequential chain to connect chain1 and chain2
story_chain = SequentialChain(
    chains = [chain1 , chain2],
    input_variables = ["topic"],
    output_variables = ["story" , "title"],
    verbose = False
)

output = story_chain.invoke({
    "topic": "Zombie attack"
})

print(output["story"])

The screams of the living mingled with the moans of the undead as the horde surged through the barricaded streets, their hunger insatiable, their pursuit relentless.  Only a handful remained, huddled in the flickering light of a dying generator, their hope as fragile as the splintered wood protecting them.


In [19]:
print(output["title"])

The Last Flicker


* Here chain1 generates a story and saves under tag "story"
* chain2 uses this story to generate a title and stores under key "title"

* verbose=False (default) → Runs quietly, you only see the final result.
* verbose=True → Prints extra details in your console, like: